## A simulation of senescence spread including a time delay

In [ ]:
import sys
sys.path.append("./Packages")
import random
import numpy as np
from functools import lru_cache
import scipy.special as sp
import itertools as it
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation
from scipy.integrate import quad
import scipy.stats as stats
from matplotlib.collections import EllipseCollection
import plotly.graph_objs as go
import plotly.express as px
import math

## Parameters

Define the parameters required in the simulation

In [ ]:
size = 100 # size of one size of the square plane under consideration in units of r_cell
r_cell = 10*10**-6 # m cell size
R_tot = 10**5 # receptor expression level
k_on = 10**8 # M^-1 min^-1 forward binding rate constant, where M =  mol/L
density = 0.35
k_e = 0.2 # min^-1 endocytosis
k_off = 0.2 # min^-1 dissociation
D_L = 10**-6 # cm^2/s diffusivity of a ligand
N_E = 2887 # number of ligands emitted per unit time (hour)
N_D = 62 #number of ligands absorbed per unit time to become senesent
delay = 0 # delay in cell becoming fully senescent
j_frac = 0.0 # fractional strength of the SASP from juxtacrine secondary senescent cells
p_frac = 1.0 # ractional strength of the SASP from paracrine secondary senescent cells
#j_prob = 1.0 # probability of neighbouring cell becoming juxtacrine senescenct
lattice = False
juxt_type = 1  # 0 is delayed juxt, 1 is imediate
verbose = False  # use with caution, leads to massive output
number_of_sims = 1 # the number of times to run each set of parameters
no_of_cells_to_run_for = 100 # the number of PARACRINE SECONDARY SENESCENT cells to run each sim for

In [ ]:
if juxt_type == 0:
    from Senesence_grid_delay import *
else:
    from Senesence_grid_delay_NOTCH import *

# convert to SI units
D_L = D_L/100**2 # now in m^2/s
k_on = k_on* 0.001/60 # now in [m^3]/[mol][s]
k_e = k_e/60
k_off = k_off/60

## Simulate the spread of senescence

This function simulates one experient where prinary senescent cells are seeded in the centre of the dish and senescence is allowed to spread. The simulation stops either when 100 paracrine secondary senescent cells have been created

In [ ]:
def one_sim(size, r_cell, R_tot, k_on, density, k_e, k_off, D_L, N_E, N_D, delay, j_frac, p_frac, j_prob, verbose, lat):
    # create instance of the class
    basic_sim = Senescence_grid(size, r_cell, R_tot, k_on, density, k_e, k_off, D_L, N_E, N_D, delay, j_frac, j_prob, p_frac, lat)
    # seed senescent cells, must provide radius inside which cells are senescent
    seed_r = 10 * r_cell
    basic_sim.seed_senescence(seed_r) # seeds cells and creates initial juxtocrine cells.
    #seed_dens = 0.1
    #basic_sim.seed_senescence_random(seed_dens)
    if verbose:
        print(len(basic_sim.scells), " senescent cells were seeded")
    # run simulation
    sim_time = delay
    #print(basic_sim.Tstruct)

    diff = 0
    t_array = [0]
    r_array = [0]
    X = []
    # time is in units of hours

    steps = 0
    while len(X) < no_of_cells_to_run_for:
        print(len(t_array))
        steps = steps + 1
        if verbose:
            print("sim time ", sim_time)
        if math.isnan(sim_time):
            break
        else:
            sim_time_new, distance_from_0 = basic_sim.one_run(verbose)
            diff = sim_time_new - sim_time
            sim_time = sim_time_new
            t_array.append(sim_time)
            r_array.append(distance_from_0)
            try:
                X, Y = zip(*basic_sim.paracrine.keys())
            except:
                X = []
        

    return t_array, basic_sim, r_array

## Run many simulations with Juxtacrine secondary senescence

In [ ]:
j_prob = 1.0
list_of_times = []

for i in range(number_of_sims):
    time_out, basic_sim, distance_out = one_sim(size, r_cell, R_tot, k_on, density, k_e, k_off, D_L, N_E, N_D, delay, j_frac, p_frac, j_prob, verbose, lattice)
    list_of_times.append(list(basic_sim.paracrine.values()))  

## Run many simulations without Juxtacrine secondary senescence

In [ ]:
j_prob = 0.0
list_of_times_no_j = []

for i in range(number_of_sims):
    time_out_no_j, basic_sim_no_j, distance_out_no_j = one_sim(size, r_cell, R_tot, k_on, density, k_e, k_off, D_L, N_E, N_D, delay, j_frac, p_frac, j_prob, verbose, lattice)
    list_of_times_no_j.append(list(basic_sim_no_j.paracrine.values()))

## Plot the senescent cells with juxtacrine secondary senescence

Plots the distribution of cells from the final simulation run, at the final time point

In [ ]:
X_in, Y_in = zip(*basic_sim.scells.keys())
Xcells_in, Ycells_in = zip(*basic_sim.sim_list)

# Ensure cells are the correct size in plot
plt.figure(figsize=[5, 5])
ax = plt.axes([0.1, 0.1, 0.8, 0.8], xlim=(-size*1000*r_cell/2., size*1000*r_cell/2.), ylim=(-size*1000*r_cell/2., size*1000*r_cell/2.))
points_whole_ax = 5 * 0.8 * 72/(size*r_cell)    # 1 point = dpi / 72 pixels
radius = r_cell
points_radius = 2 * radius / 1.0 * points_whole_ax

# Plot the juxtacrine secondary senescent cells
try:
    X1_in, Y1_in = zip(*basic_sim.jux_scells.keys())
    X1 = [x * 1000 for x in X1_in]
    Y1 = [x * 1000 for x in Y1_in]
    ax.scatter(X1, Y1, s=points_radius**2, label = "Juxtacrine senescent cells", color = "red", edgecolors='black')
except:
    print("No Juxtacrine cells")
    
# Plot the paracrine secondary senescent cells
try:
    X2_in, Y2_in = zip(*basic_sim.paracrine.keys())
    X2 = [x * 1000 for x in X2_in]
    Y2 = [x * 1000 for x in Y2_in]
    ax.scatter(X2, Y2, s=points_radius**2, label = "Paracrine senescent cells", color = "blue", edgecolors='black')
except:
    print("No Paracrine cells")
    

# Find how long cells have to wait to become fully senescent
waiting_cells = basic_sim.Tstruct.values()
waiting_cell_times = list(basic_sim.Tstruct.keys())
waiting_juxt = []
waiting_para = []
waiting_juxt_time = []
waiting_para_time = []

# Plot these "waiting" cells
for i, cell_group in enumerate(waiting_cells):

    if cell_group[1] == 1:

        waiting_para = waiting_para + cell_group[0]
        waiting_para_time = waiting_para_time + [waiting_cell_times[i]]
            
    else:

        waiting_juxt = waiting_juxt + cell_group[0]
        waiting_juxt_time = waiting_juxt_time + [waiting_cell_times[i]]*len(cell_group[0])
         

try:
    X3_in, Y3_in = zip(*waiting_para)
    X3 = [x * 1000 for x in X3_in]
    Y3 = [x * 1000 for x in Y3_in]
    ax.scatter(X3, Y3, s=points_radius**2, label = "Cells waiting to become paracrine senescent", c = waiting_para_time, edgecolors='black')
except:
    print("No cells waiting to become paracrine senescent")
    
try:
    X4_in, Y4_in = zip(*waiting_juxt)
    X4 = [x * 1000 for x in X4_in]
    Y4 = [x * 1000 for x in Y4_in]
    ax.scatter(X4, Y4, s=points_radius**2, label = "Cells waiting to become juxtacrine senescent", c = "orange",edgecolors='black')#, c = waiting_juxt_time, cmap = "cool")
except:
    print("No cells waiting to become juxtacrine senescent")
    
    
# Plot non-senescent cells
X = [x * 1000 for x in X_in]
Y = [x * 1000 for x in Y_in]
    
ax.spines['bottom'].set_linewidth(3)
ax.spines['top'].set_linewidth(3)
ax.spines['right'].set_linewidth(3)
ax.spines['left'].set_linewidth(3)

Xcells = [x * 1000 for x in Xcells_in]
Ycells = [x * 1000 for x in Ycells_in]

ax.scatter(X, Y, s=points_radius**2, label = "Seeded senescent cells", color = "black", edgecolors='black')
ax.scatter(Xcells, Ycells, s=points_radius**2, label = "Seeded senescent cells", color = "white", edgecolors='black')
    
plt.title("Without juxtacrine signalling", {'size':'14'})
for label in (ax.get_xticklabels() + ax.get_yticklabels()):
    label.set_fontsize(13)
plt.xlabel("(mm)", {'size':'14'})
plt.ylabel("(mm)", {'size':'14'})
plt.legend(bbox_to_anchor=(1.04, 1), loc="upper left")
plt.show()

## Plot senescent cells without juxtacrine

In [ ]:
X_in, Y_in = zip(*basic_sim_no_j.scells.keys())
Xcells_in, Ycells_in = zip(*basic_sim_no_j.sim_list)

plt.figure(figsize=[5, 5])
ax = plt.axes([0.1, 0.1, 0.8, 0.8], xlim=(-size*1000*r_cell/2., size*1000*r_cell/2.), ylim=(-size*1000*r_cell/2., size*1000*r_cell/2.))
points_whole_ax = 5 * 0.8 * 72/(size*r_cell)    # 1 point = dpi / 72 pixels
radius = r_cell
points_radius = 2 * radius / 1.0 * points_whole_ax


try:
    X1_in, Y1_in = zip(*basic_sim_no_j.jux_scells.keys())
    X1 = [x * 1000 for x in X1_in]
    Y1 = [x * 1000 for x in Y1_in]
    ax.scatter(X1, Y1, s=points_radius**2, label = "Juxtacrine senescent cells", color = "red", edgecolors='black')
except:
    print("No Juxtacrine cells")
    
try:
    X2_in, Y2_in = zip(*basic_sim_no_j.paracrine.keys())
    X2 = [x * 1000 for x in X2_in]
    Y2 = [x * 1000 for x in Y2_in]
    ax.scatter(X2, Y2, s=points_radius**2, label = "Paracrine senescent cells", color = "blue", edgecolors='black')
except:
    print("No Paracrine cells")
    

waiting_cells = basic_sim_no_j.Tstruct.values()
waiting_cell_times = list(basic_sim_no_j.Tstruct.keys())
waiting_juxt = []
waiting_para = []
waiting_juxt_time = []
waiting_para_time = []

for i, cell_group in enumerate(waiting_cells):
    if cell_group[1] == 1:
        waiting_para = waiting_para + cell_group[0]
        waiting_para_time = waiting_para_time + [waiting_cell_times[i]]
            
    else:
        waiting_juxt = waiting_juxt + cell_group[0]
        waiting_juxt_time = waiting_juxt_time + [waiting_cell_times[i]]*len(cell_group[0])
         

try:
    X3_in, Y3_in = zip(*waiting_para)
    X3 = [x * 1000 for x in X3_in]
    Y3 = [x * 1000 for x in Y3_in]
    ax.scatter(X3, Y3, s=points_radius**2, label = "Cells waiting to become paracrine senescent", c = waiting_para_time, edgecolors='black')
except:
    print("No cells waiting to become paracrine senescent")
    
try:
    X4_in, Y4_in = zip(*waiting_juxt)
    X4 = [x * 1000 for x in X4_in]
    Y4 = [x * 1000 for x in Y4_in]
    ax.scatter(X4, Y4, s=points_radius**2, label = "Cells waiting to become juxtacrine senescent", c = "orange", edgecolors='black')#, c = waiting_juxt_time, cmap = "cool")
except:
    print("No cells waiting to become juxtacrine senescent")
   

X = [x * 1000 for x in X_in]
Y = [x * 1000 for x in Y_in]

Xcells = [x * 1000 for x in Xcells_in]
Ycells = [x * 1000 for x in Ycells_in]
    
ax.spines['bottom'].set_linestyle('dotted')
ax.spines['top'].set_linestyle('dotted')
ax.spines['right'].set_linestyle('dotted')
ax.spines['left'].set_linestyle('dotted')
    
ax.spines['bottom'].set_linewidth(3)
ax.spines['top'].set_linewidth(3)
ax.spines['right'].set_linewidth(3)
ax.spines['left'].set_linewidth(3)
    
ax.scatter(X, Y, s=points_radius**2, label = "Seeded senescent cells", color = "black", edgecolors='black')
ax.scatter(Xcells, Ycells, s=points_radius**2, label = "Seeded senescent cells", color = "white", edgecolors='black')
    
plt.title("Without juxtacrine signalling", {'size':'14'})
for label in (ax.get_xticklabels() + ax.get_yticklabels()):
    label.set_fontsize(13)
plt.xlabel("(mm)", {'size':'14'})
plt.ylabel("(mm)", {'size':'14'})
plt.legend(bbox_to_anchor=(1.04, 1), loc="upper left")
plt.show()

## Plot the timescale of senescence induction 

In [ ]:
fig = go.Figure()

for i in range(len(list_of_times)):
    y_vals = np.arange(0,len(list_of_times[i]))
    if delay!=0:
        x_vals = list_of_times[i]
    else:
        x_vals = [item for sublist in list_of_times[i] for item in sublist]


    fig.add_trace(go.Scatter(x = x_vals, y = y_vals, line_color = "black"))


for i in range(len(list_of_times_no_j)):
    y_vals = np.arange(0,len(list_of_times_no_j[i]))
    if delay!=0:
        x_vals = list_of_times_no_j[i]
    else:
        x_vals = [item for sublist in list_of_times_no_j[i] for item in sublist]
    fig.add_trace(go.Scatter(x = x_vals, y = y_vals,line=dict(color="black", dash='dot')))


fig.add_trace(go.Scatter(x = 48*np.ones(50), y = np.arange(-10, 210, 250/50),  line=dict(color="grey", dash='dash'), mode = "lines"))
fig.add_trace(go.Scatter(x = 48*2*np.ones(50), y = np.arange(-10, 210, 250/50),  line=dict(color="grey", dash='dash'), mode = "lines"))
fig.add_trace(go.Scatter(x = 48*3*np.ones(50), y = np.arange(-10, 210, 250/50),  line=dict(color="grey", dash='dash'), mode = "lines"))
fig.add_trace(go.Scatter(x = 48*4*np.ones(50), y = np.arange(-10, 210, 250/50),  line=dict(color="grey", dash='dash'), mode = "lines"))
fig.add_trace(go.Scatter(x = 48*5*np.ones(50), y = np.arange(-10, 210, 250/50),  line=dict(color="grey", dash='dash'), mode = "lines"))
fig.add_trace(go.Scatter(x = 48*6*np.ones(50), y = np.arange(-10, 210, 250/50),  line=dict(color="grey", dash='dash'), mode = "lines"))
fig.add_trace(go.Scatter(x = 48*7*np.ones(50), y = np.arange(-10, 210, 250/50),  line=dict(color="grey", dash='dash'), mode = "lines"))
fig.update_layout(xaxis_title = "Time (h)", yaxis_title = "Number of paracrine senescent cells", font=dict(size=18))
fig.update_yaxes(title_font=dict(size=20), type = "log", tickvals = [1, 10,  100])
fig.update_xaxes(title_font=dict(size=20), type = "log")
fig.show()

## Create animations

In [ ]:
from matplotlib import animation

In [ ]:
def ani_func(num, frame_times):   
    
    step = frame_times.index(num)
    if step>0:
        
        try:
            X1_in, Y1_in = zip(*[coord for coord, time in basic_sim.jux_scells.items() if time == list(basic_sim.paracrine.values())[step][0]])
            X1 = [x * 1000 for x in X1_in]
            Y1 = [x * 1000 for x in Y1_in]
            ax.scatter(X1, Y1, s=points_radius**2, label = "Juxtacrine senescent cells", color = "red", edgecolors='black')
        except:
            print("No Juxtacrine cells")


        try:
            X2_in, Y2_in = zip(list(basic_sim.paracrine.keys())[step])
            X2 = [x * 1000 for x in X2_in]
            Y2 = [x * 1000 for x in Y2_in]
            ax.scatter(X2, Y2, s=points_radius**2, label = "Paracrine senescent cells", color = "blue", edgecolors='black')
        except:
            print("No Paracrine cells")
            
    else:
        try:
            X1_in, Y1_in = zip(*[coord for coord, time in basic_sim.jux_scells.items() if time == 0])
            X1 = [x * 1000 for x in X1_in]
            Y1 = [x * 1000 for x in Y1_in]
            ax.scatter(X1, Y1, s=points_radius**2, label = "Juxtacrine senescent cells", color = "red", edgecolors='black')
        except:
            print("No Juxtacrine cells")
            
def ani_func_no_j(num, frame_times):   
    
    step = frame_times.index(num)
    if step>0:
        
        try:
            X1_in, Y1_in = zip(*[coord for coord, time in basic_sim_no_j.jux_scells.items() if time == list(basic_sim_no_j.paracrine.values())[step][0]])
            X1 = [x * 1000 for x in X1_in]
            Y1 = [x * 1000 for x in Y1_in]
            ax.scatter(X1, Y1, s=points_radius**2, label = "Juxtacrine senescent cells", color = "red", edgecolors='black')
        except:
            print("No Juxtacrine cells")


        try:
            X2_in, Y2_in = zip(list(basic_sim_no_j.paracrine.keys())[step])
            X2 = [x * 1000 for x in X2_in]
            Y2 = [x * 1000 for x in Y2_in]
            ax.scatter(X2, Y2, s=points_radius**2, label = "Paracrine senescent cells", color = "blue", edgecolors='black')
        except:
            print("No Paracrine cells")
            
    else:
        try:
            X1_in, Y1_in = zip(*[coord for coord, time in basic_sim_no_j.jux_scells.items() if time == 0])
            X1 = [x * 1000 for x in X1_in]
            Y1 = [x * 1000 for x in Y1_in]
            ax.scatter(X1, Y1, s=points_radius**2, label = "Juxtacrine senescent cells", color = "red", edgecolors='black')
        except:
            print("No Juxtacrine cells")


In [ ]:
X_in, Y_in = zip(*basic_sim.scells.keys())

time = list(basic_sim.paracrine.values())
time = [i[0] for i in time]
time.insert(0, 0)
frame_times = [j-i for i, j in zip(time[:-1], time[1:])]
frame_times.append(30)

frame_t = []
for i, item in enumerate(frame_times):
    rnd = int(np.floor(item*1))
    if rnd > 0:
        frame_t.extend([item] * rnd)
    else:
        frame_t.extend([item] * 1)

fig = plt.figure(figsize=[5, 5])
ax = plt.axes([0.1, 0.1, 0.8, 0.8], xlim=(-size*1000*r_cell/2., size*1000*r_cell/2.), ylim=(-size*1000*r_cell/2., size*1000*r_cell/2.))
points_whole_ax = 5 * 0.8 * 72/(size*r_cell)    # 1 point = dpi / 72 pixels
radius = r_cell
points_radius = 2 * radius / 1.0 * points_whole_ax

X = [x * 1000 for x in X_in]
Y = [x * 1000 for x in Y_in]

ax.spines['bottom'].set_linewidth(3)
ax.spines['top'].set_linewidth(3)
ax.spines['right'].set_linewidth(3)
ax.spines['left'].set_linewidth(3)


ax.scatter(X, Y, s=points_radius**2, label = "Seeded senescent cells", color = "black", edgecolors='black')

plt.title("With juxtacrine signalling", {'size':'14'})
for label in (ax.get_xticklabels() + ax.get_yticklabels()):
    label.set_fontsize(13)
plt.xlabel("(mm)", {'size':'14'})
plt.ylabel("(mm)", {'size':'14'})


line_ani = animation.FuncAnimation(fig, ani_func, frames=frame_t, fargs=(frame_times,), interval=200)
f = r"/home/lucymartin/Desktop/animate_func_juxt.gif"
writergif = animation.PillowWriter(fps=len(frame_t)/6)
line_ani.save(f, writer=writergif)

In [ ]:
X_in, Y_in = zip(*basic_sim_no_j.scells.keys())

time_no_j = list(basic_sim_no_j.paracrine.values())
time_no_j = [i[0] for i in time_no_j]
time_no_j.insert(0, 0)
frame_times_no_j = [j-i for i, j in zip(time_no_j[:-1], time_no_j[1:])]
frame_times_no_j.append(30)

frame_t_no_j = []
for i, item in enumerate(frame_times_no_j):
    rnd = int(np.floor(item*1))
    if rnd > 0:
        frame_t_no_j.extend([item] * rnd)
    else:
        frame_t_no_j.extend([item] * 1)

fig = plt.figure(figsize=[5, 5])
ax = plt.axes([0.1, 0.1, 0.8, 0.8], xlim=(-size*1000*r_cell/2., size*1000*r_cell/2.), ylim=(-size*1000*r_cell/2., size*1000*r_cell/2.))
points_whole_ax = 5 * 0.8 * 72/(size*r_cell)    # 1 point = dpi / 72 pixels
radius = r_cell
points_radius = 2 * radius / 1.0 * points_whole_ax

X = [x * 1000 for x in X_in]
Y = [x * 1000 for x in Y_in]

ax.spines['bottom'].set_linewidth(3)
ax.spines['top'].set_linewidth(3)
ax.spines['right'].set_linewidth(3)
ax.spines['left'].set_linewidth(3)


ax.scatter(X, Y, s=points_radius**2, label = "Seeded senescent cells", color = "black", edgecolors='black')

plt.title("Without juxtacrine signalling", {'size':'14'})
for label in (ax.get_xticklabels() + ax.get_yticklabels()):
    label.set_fontsize(13)
plt.xlabel("(mm)", {'size':'14'})
plt.ylabel("(mm)", {'size':'14'})

line_ani = animation.FuncAnimation(fig, ani_func_no_j, frames=frame_t_no_j, fargs=(frame_times_no_j,), interval=200)
f = r"/home/lucymartin/Desktop/animate_func_no_j.gif"
writergif = animation.PillowWriter(fps=len(frame_t)/6)
line_ani.save(f, writer=writergif)